In [ ]:
from keras.layers import Input, merge
from keras.models import Model,Sequential
from layers import VGGNormalize,ReflectionPadding2D,Denormalize,conv_bn_relu,res_conv,dconv_bn_relu
from loss import dummy_loss,StyleReconstructionRegularizer,FeatureReconstructionRegularizer,TVRegularizer

import nets

%matplotlib inline

%reload_ext autoreload

%autoreload 2


* Input 3 × 256 × 256
* Reflection Padding (40 × 40) 3 × 336 × 336
* 32 × 9 × 9 conv, stride 1 32 × 336 × 336
* 64 × 3 × 3 conv, stride 2 64 × 168 × 168
* 128 × 3 × 3 conv, stride 2 128 × 84 × 84
* Residual block, 128 filters 128 × 80 × 80
* Residual block, 128 filters 128 × 76 × 76
* Residual block, 128 filters 128 × 72 × 72
* Residual block, 128 filters 128 × 68 × 68
* Residual block, 128 filters 128 × 64 × 64
* 64 × 3 × 3 conv, stride 1/2 64 × 128 × 128
* 32 × 3 × 3 conv, stride 1/2 32 × 256 × 256
* 3 × 9 × 9 conv, stride 1 3 × 256 × 256

Image Util

In [9]:



model = nets.image_transform_net()
# print model.summary() 

#if tranning

model = nets.loss_net(model.output,model.input)

print model.summary()


VGG Model weights loaded.
Getting style features from VGG network.
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 256, 256, 3)   0                                            
____________________________________________________________________________________________________
reflectionpadding2d_3 (Reflectio (None, 336, 336, 3)   0           input_3[0][0]                    
____________________________________________________________________________________________________
convolution2d_33 (Convolution2D) (None, 336, 336, 32)  7808        reflectionpadding2d_3[0][0]      
____________________________________________________________________________________________________
batchnormalization_27 (BatchNorm (None, 336, 336, 32)  128         convolution2d_33[0][0]           
________________________

In [ ]:
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import time
import numpy as np
from keras.callbacks import TensorBoard

img_width = 256
img_height = 256
nb_epoch = 20
train_batchsize = 1 #4
train_image_path = "images/train/"

learning_rate = 1e-3 #1e-3
optimizer = Adam(lr=learning_rate,beta_1=0.99)

model.compile(optimizer,  dummy_loss)  # Dummy loss since we are learning from regularizes

datagen = ImageDataGenerator(rescale=1. / 255)

dummy_y = np.zeros((train_batchsize, img_height, img_width, 3)) # Dummy output, not used since we use regularizers to train

tbCallBack = TensorBoard(log_dir='/Graph', histogram_freq=0, write_graph=True, write_images=True)

for i in range(nb_epoch):
    print("Epoch : %d" % (i + 1))

    for x in datagen.flow_from_directory(train_image_path, class_mode=None, batch_size=train_batchsize,
        target_size=(img_width, img_height), shuffle=False):    
        
        print("train image ")
        t1 = time.time()
        hist = model.fit(x, dummy_y, batch_size=train_batchsize, nb_epoch=1, verbose=1,callbacks=[tbCallBack])
        loss = hist.history['loss'][0]
      
        print(loss,(time.time() -t1))
    

Epoch : 1
Found 4 images belonging to 1 classes.
train image 
Epoch 1/1
1/1 [==============================] - 23s - loss: 425665284828296411873280.0000
(4.2566528482829641e+23, 76.22673082351685)
train image 
Epoch 1/1
1/1 [==============================] - 25s - loss: 425665284828296411873280.0000
(4.2566528482829641e+23, 66.78861713409424)
train image 
